In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
!CUDA_VISIBLE_DEVICES=3 python3.7 ./ner/get_word_details.py --config SW2k.8k.2000k.NoLM --model Repository/NERModel/SW2k.8k.2000k.NoLM.86.model --out error_analisys/SW2k.8k.2000k.NoLM.csv


2019/06/27 14:55:54
/sentencepiece/src/normalizer.cc(33) LOG(INFO) precompiled_charsmap is empty. use identity normalization.
/sentencepiece/src/normalizer.cc(33) LOG(INFO) precompiled_charsmap is empty. use identity normalization.
=========== Load train data ===========
 19%|█████▊                         | 169978/903738 [00:00<00:02, 342722.41it/s]^C

Traceback (most recent call last):
  File "./ner/get_word_details.py", line 89, in <module>
    train_word_documents, train_label_documents = load_seq_data(os.path.join(config_dic.get("ner_input_dir"), "train.bioes"), config_dic.get("number_normalize"))
  File "./ner/train.py", line 47, in load_seq_data
    word, label = line.split(" ")
KeyboardInterrupt


In [3]:
!CUDA_VISIBLE_DEVICES=3 python3.7 ./ner/get_word_details.py --config BL.2000k.NoLM --model Repository/NERModel/BL.2000k.NoLM.77.model --out error_analisys/BL.2000k.NoLM.csv

2019/06/27 14:55:57
=========== Load train data ===========
 52%|████████████████▏              | 470296/903738 [00:01<00:01, 419363.06it/s]^C

Traceback (most recent call last):
  File "./ner/get_word_details.py", line 89, in <module>
    train_word_documents, train_label_documents = load_seq_data(os.path.join(config_dic.get("ner_input_dir"), "train.bioes"), config_dic.get("number_normalize"))
  File "./ner/train.py", line 47, in load_seq_data
    word, label = line.split(" ")
KeyboardInterrupt


# BLで抽出できて、SWで抽出できなかったもの。

In [4]:
bl_df = pd.read_csv("./error_analisys/BL.2000k.NoLM.csv.csv", index_col=0)
sw_df = pd.read_csv("./error_analisys/SW2k.8k.2000k.NoLM.csv.csv", index_col=0)

In [5]:
bl_df["precision"] = bl_df["tp"] / (bl_df["tp"] + bl_df["fp"])
bl_df["recall"] = bl_df["tp"] / (bl_df["tp"] + bl_df["fn"])

In [6]:
sw_df["precision"] = sw_df["tp"] / (sw_df["tp"] + sw_df["fp"])
sw_df["recall"] = sw_df["tp"] / (sw_df["tp"] + sw_df["fn"])

In [7]:
merged_df = pd.merge(left=bl_df, right=sw_df, on="entity", suffixes=["_bl", "_sw"])

In [8]:
merged_df["tp_bl-tp_sw"] = merged_df.tp_bl - merged_df.tp_sw

In [9]:
merged_df[["entity", "true_count_bl", "Unknown_bl", "precision_bl", "recall_bl", "precision_sw", "recall_sw", "SW2k", "SW8k", "tp_bl-tp_sw"]].sort_values("tp_bl-tp_sw", ascending=False).sum()

entity           PDMSphenolicpicrosides I and IIC0Cl0AsCITAM000...
true_count_bl                                                25347
Unknown_bl                                                     906
precision_bl                                               6155.56
recall_bl                                                  6279.16
precision_sw                                               6196.24
recall_sw                                                  6331.28
SW2k             ['▁P D MS']['▁ phen ol ic']['▁p ic ro s ide s'...
SW8k             ['▁PD MS']['▁phenolic']['▁pi cr oside s', '▁I'...
tp_bl-tp_sw                                                   -135
dtype: object

In [16]:
merged_df.sort_values("fn_bl", ascending=False)

,entity,true_count_bl,pred_count_bl,tp_bl,fp_bl,fn_bl,Unknown_bl,precision_bl,recall_bl,true_count_sw,pred_count_sw,tp_sw,fp_sw,fn_sw,Unknown_sw,SW2k,SW8k,precision_sw,recall_sw,tp_bl-tp_sw
121,NAC,27,46,27,0,19,False,1.0,0.586957,27,45,27,0,18,False,['▁N A C'],['▁N AC'],1.0,0.600000,0
128,PAH,26,45,26,0,19,False,1.0,0.577778,26,45,26,0,19,False,['▁P A H'],['▁PAH'],1.0,0.577778,0
3248,silymarin,1,15,1,0,14,False,1.0,0.066667,1,15,1,0,14,False,['▁sil ym ar in'],['▁sil ym arin'],1.0,0.066667,0
7761,GMP,0,11,0,0,11,False,NaN,0.000000,0,11,0,0,11,False,['▁G MP'],['▁G MP'],NaN,0.000000,0
8242,HFCS,0,11,0,0,11,False,NaN,0.000000,0,11,0,0,11,False,['▁H F C S'],['▁H F CS'],NaN,0.000000,0
4406,MPP,1,11,1,0,10,False,1.0,0.090909,1,10,1,0,9,False,['▁ MP P'],['▁M PP'],1.0,0.100000,0
8226,sDNA,0,10,0,0,10,False,NaN,0.000000,0,2,0,0,2,False,['▁ s DNA'],['▁s DNA'],NaN,0.000000,0
8288,Chl,0,10,0,0,10,False,NaN,0.000000,0,7,0,0,7,False,['▁Ch l'],['▁Ch l'],NaN,0.000000,0
7882,HemoHIM,0,10,0,0,10,False,NaN,0.000000,0,10,0,0,10,False,['▁H e mo H I M'],['▁He mo H IM'],NaN,0.000000,0
7898,QAS,0,10,0,0,10,False,NaN,0.000000,0,9,0,0,9,False,['▁ Q A S'],['▁Q AS'],NaN,0.000000,0


# 未知語に対して、BLとSWで優位性はあったか。

In [50]:
unknown_merged_df = merged_df[merged_df["Unknown_bl"]]
unknown_merged_df[["entity", "true_count_bl", "Unknown_bl", "precision_bl", "recall_bl", "precision_sw", "recall_sw", "SW2k", "SW8k", "tp_bl-tp_sw"]].sort_values("tp_bl-tp_sw", ascending=False)

,entity,true_count_bl,Unknown_bl,precision_bl,recall_bl,precision_sw,recall_sw,SW2k,SW8k,tp_bl-tp_sw
1362,P ( MAGP - co - DMAEMA ) - b - PPDSMA,4,True,0.750000,1.0,0.000000,NaN,"['▁P', '▁(', '▁M A G P', '▁-', '▁co', '▁-', '▁...","['▁P', '▁(', '▁M AG P', '▁-', '▁co', '▁-', '▁ ...",3
1275,yuwen00f0,4,True,1.000000,1.0,0.250000,1.0,['▁ y u w en 00 f 0'],['▁ y u w en 00 f 0'],3
1306,0OXY ( Gly ) ( 0 ),4,True,0.500000,1.0,0.000000,NaN,"['▁ 0 OX Y', '▁(', '▁G ly', '▁)', '▁(', '▁0', ...","['▁0 OX Y', '▁(', '▁Gly', '▁)', '▁(', '▁0', '▁)']",2
3011,Yuwen00f0,2,True,1.000000,1.0,0.000000,NaN,['▁ Y u w en 00 f 0'],['▁Y u w en 00 f 0'],2
1665,rapta - C,3,True,0.666667,1.0,0.000000,NaN,"['▁r ap ta', '▁-', '▁C']","['▁ rap ta', '▁-', '▁C']",2
2677,AuTTF,2,True,1.000000,1.0,0.000000,NaN,['▁A u T T F'],['▁Au T TF'],2
2373,Myritol 000,2,True,1.000000,1.0,0.000000,NaN,"['▁M yr it ol', '▁ 000']","['▁M y rit ol', '▁ 000']",2
2627,epi - mukulin,2,True,1.000000,1.0,0.000000,NaN,"['▁epi', '▁-', '▁mu k ul in']","['▁epi', '▁-', '▁mu k ulin']",2
6952,luteolin O - acylhexoside,1,True,1.000000,1.0,0.000000,NaN,"['▁ lu te ol in', '▁O', '▁-', '▁a c yl h ex os...","['▁l ute olin', '▁O', '▁-', '▁acyl hex oside']",1
2804,guanide,2,True,1.000000,1.0,0.500000,1.0,['▁ gu an ide'],['▁ guan ide'],1


In [47]:
unknown_merged_df.describe()

,true_count_bl,pred_count_bl,tp_bl,fp_bl,fn_bl,precision_bl,recall_bl,true_count_sw,pred_count_sw,tp_sw,fp_sw,fn_sw,precision_sw,recall_sw,tp_bl-tp_sw
count,906.000000,906.000000,906.000000,906.000000,906.000000,808.000000,741.000000,906.000000,906.000000,906.00000,906.000000,906.000000,808.000000,744.000000,906.000000
mean,1.129139,0.981236,0.827815,0.301325,0.153422,0.773309,0.866397,1.129139,0.975717,0.82340,0.305740,0.152318,0.777991,0.864023,0.004415
std,0.763744,0.668701,0.652356,0.715062,0.521022,0.405762,0.339462,0.763744,0.649237,0.64777,0.726993,0.487388,0.404015,0.340094,0.443470
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,-4.000000
25%,1.000000,1.000000,0.000000,0.000000,0.000000,0.750000,1.000000,1.000000,1.000000,0.00000,0.000000,0.000000,1.000000,1.000000,0.000000
50%,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.00000,0.000000,0.000000,1.000000,1.000000,0.000000
75%,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.00000,0.000000,0.000000,1.000000,1.000000,0.000000
max,8.000000,5.000000,4.000000,8.000000,5.000000,1.000000,1.000000,8.000000,4.000000,4.00000,8.000000,4.000000,1.000000,1.000000,3.000000


In [63]:
merged_df[merged_df["entity"].map(lambda x: x in "tetrazolium")]

NameError: name 'meged_df' is not defined

# SW分割したおかげで、結果が良くなったと思われる例。

# SW分割したせいで、結果が悪くなった例